### Using OpenAI like Developer

1. Getting started - env variables
2. Using OpenAI library - client, roles
3. Prompting - few shot, chain of thought


Note: Get OpenAI key from - https://platform.openai.com/api-keys

## 1. Getting Started

First load the [OpenAI Python Library](https://github.com/openai/openai-python/tree/main)!

In [ ]:
# Install the dependencies and get OpenAI API Key from the secrets.
!pip install openai -qU

#### Setting Environment Variables

In [ ]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key")

## 2. Using Client


The core feature of the OpenAI Python Library is the `OpenAI()` client. It's how we're going to interact with OpenAI's models.

> NOTE: You can reference OpenAI's [documentation](https://platform.openai.com/docs/api-reference/authentication?lang=python) whenever you get stuck, have questions, or want to dive deeper.

In [ ]:
!which openai

In [ ]:
import openai

In [ ]:
from openai import OpenAI

openai_client = OpenAI()

Now that we have our client - we're going to use the .chat.completions.create method to interact with the gpt-4o model.

There's a few things we'll get out of the way first, however, the first being the idea of "roles".

First it's important to understand the object that we're going to use to interact with the endpoint. It expects us to send an array of objects of the following format:

In [ ]:
{"role" : "ROLE", "content" : "YOUR CONTENT HERE", "name" : "THIS IS OPTIONAL"}

Second, there are three "roles" available to use to populate the `"role"` key:

- `system`
- `assistant`
- `user`

OpenAI provides some context for these roles [here](https://help.openai.com/en/articles/7042661-moving-from-completions-to-chat-completions-in-the-openai-api).

We'll explore these roles in more depth as they come up - but for now we're going to just stick with the basic role `user`. The `user` role is, as it would seem, the user!

Thirdly, it expects us to specify a model!

We'll use the `gpt-4o` or `gpt-4o-mini` model as stated above.

Let's look at an example!


In [ ]:
response = openai_client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role" : "user", "content" : "Hello, how are you?"}]
)

In [ ]:
response

Helper functions defined to aid using OpenAI API - for easier


In [ ]:
from IPython.display import display, Markdown

def get_response(client: OpenAI, messages: list, model: str = "gpt-4o") -> str:
    return client.chat.completions.create(
        model=model,
        messages=messages
    )

def system_prompt(message: str) -> dict:
    return {"role": "system", "content": message}

def assistant_prompt(message: str) -> dict:
    return {"role": "assistant", "content": message}

def user_prompt(message: str) -> dict:
    return {"role": "user", "content": message}

def pretty_print(message: str) -> str:
    display(Markdown(message.choices[0].message.content))

In [ ]:
# test the helper functions

YOUR_PROMPT = "Hello, how are you?"
messages_list = [user_prompt(YOUR_PROMPT)]

chatgpt_response = get_response(openai_client, messages_list)

pretty_print(chatgpt_response)

## 2. Roles

Now we can extend our prompts to include a system prompt.

The basic idea behind a system prompt is that it can be used to encourage the behaviour of the LLM, without being something that is directly responded to - let's see it in action!

In [ ]:
list_of_prompts = [
    system_prompt("You are irate and extremely hungry. Feel free to express yourself."),
    user_prompt("Do you prefer crushed ice or cubed ice?")
]

irate_response = get_response(openai_client, list_of_prompts)
pretty_print(irate_response)

As you can see - the response we get back is very much as directed in the system prompt!

Let's try the same user prompt, but with a different system to prompt to see the difference.

In [ ]:
list_of_prompts = [
    system_prompt("You are joyful and having the best day. Please act like a person in that state of mind."),
    user_prompt("Do you prefer crushed ice or cubed ice?")
]

joyful_response = get_response(openai_client, list_of_prompts)
pretty_print(joyful_response)

With a simple modification of the system prompt - you can see that get completely different behaviour, and that's the main goal of prompt engineering as a whole.

Congratulations, you created your first prompt!

## 3. Few shot prompting

Now that we have a basic handle on the `system` role and the `user` role - let's examine what we might use the `assistant` role for.

The most common usage pattern is to "pretend" that we're answering our own questions. This helps us further guide the model toward our desired behaviour. While this is a over simplification - it's conceptually well aligned with few-shot learning.

First, we'll try and "teach" `gpt-4o` some nonsense words as was done in the paper ["Language Models are Few-Shot Learners"](https://arxiv.org/abs/2005.14165).

In [ ]:
list_of_prompts = [
    user_prompt("Please use the words 'stimple' and 'falbean' in a sentence.")
]

stimple_response = get_response(openai_client, list_of_prompts)
pretty_print(stimple_response)

The example shows how assistant role guides the final result sentence.

## 4. Chain of Thought Prompting

We'll head one level deeper and explore the world of Chain of Thought prompting (CoT).

This is a process by which we can encourage the LLM to handle slightly more complex tasks.

Let's look at a simple reasoning based example without CoT.

In [ ]:
reasoning_problem = """
Billy wants to get home from San Fran. before 7PM EDT.

It's currently 1PM local time.

Billy can either fly (3hrs), and then take a bus (2hrs), or Billy can take the teleporter (0hrs) and then a bus (1hrs).

Does it matter which travel option Billy selects?
"""

list_of_prompts = [
    user_prompt(reasoning_problem)
]

reasoning_response = get_response(openai_client, list_of_prompts)
pretty_print(reasoning_response)

Let's see if we can leverage a simple CoT prompt to improve our model's performance on this task:

In [ ]:
list_of_prompts = [
    user_prompt(reasoning_problem + " Think though your response step by step.")
]

reasoning_response = get_response(openai_client, list_of_prompts)
pretty_print(reasoning_response)

## 5. Prompt Engineering Principles

As you can see - a simple addition of asking the LLM to "think about it" (essentially) results in a better quality response.

There's a [great paper](https://arxiv.org/pdf/2312.16171v1.pdf) that dives into some principles for effective prompt generation.

Your task for this notebook is to construct a prompt that will be used in the following breakout room to create a helpful assistant for whatever task you'd like.

## 6. Test the prompt with using the LLM-as-a-judge

In [ ]:
system_template = """\
WRITE YOUR SYSTEM PROMPT HERE
"""


In [ ]:
user_template = """{input}
WRITE YOUR USER PROMPT HERE
"""

In [ ]:
system_template = """\
You are the best chef in the world and you are sharing your best recipes Answer customer's questions in a polite way. Provide answer in JSON format.
"""

In [ ]:
user_template = """{input}
How long does it take to bake a cake?
Calculate the result by summing up minutes on individual tasks.
"""

In [ ]:
query = "It takes 30  minute to mix dough and 20 minutes to bake it in the oven."


list_of_prompts = [
    system_prompt(system_template),
    user_prompt(user_template.format(input=query))
]

test_response = get_response(openai_client, list_of_prompts)

pretty_print(test_response)

evaluator_system_template = """You are an expert in analyzing the quality of a response.

You should be hyper-critical.

Provide scores (out of 10) for the following attributes:

1. Clarity - how clear is the response
2. Faithfulness - how related to the original query is the response
3. Correctness - was the response correct?

Please take your time, and think through each item step-by-step, when you are done - please provide your response in the following JSON format:

{"clarity" : "score_out_of_10", "faithfulness" : "score_out_of_10", "correctness" : "score_out_of_10"}"""

evaluation_template = """Query: {input}
Response: {response}"""

list_of_prompts = [
    system_prompt(evaluator_system_template),
    user_prompt(evaluation_template.format(
        input=query,
        response=test_response.choices[0].message.content
    ))
]

evaluator_response = openai_client.chat.completions.create(
    model="gpt-4o",
    messages=list_of_prompts,
    response_format={"type" : "json_object"}
)

In [ ]:
pretty_print(evaluator_response)

Completed!